# Notebook for massaging different sources of text into a unified set of small .txt documents

## Roam Research

In [ ]:
# https://stackoverflow.com/questions/21028979/recursive-iteration-through-nested-json-for-specific-key-in-python
def item_generator(json_input, lookup_key):
     if isinstance(json_input, dict):
         for k, v in json_input.items():
             if k == lookup_key:
                 yield v
             else:
                 yield from  item_generator(v, lookup_key)
     elif isinstance(json_input, list):
         for item in json_input:
             yield from item_generator(item, lookup_key)

In [ ]:
# Roam Research
import json , re
with open('YOUR_ROAM_GRAPH.json') as json_file:
     data = json.load(json_file)

file2 = open("roam_research.txt" , "a")
for st in item_generator(data , "string"):
    stripped_st = st.replace('{{[[r/moved]]}}',''); # repeat these two steps with all the formatting you want to remove
    stripped_st = re.sub(r'\(\(.*\)\)' , '' , stripped_st);   
    if len(stripped_st) > 0: # if there's anything left afterwards, add it to the output
        stripped_st = stripped_st + '\n';
        file2.write(stripped_st)
        
file2.close()

split into chunks for better training

In [ ]:
import os
!mkdir roam_files
os.system('split -l 100 roam_research.txt roam_files/splitroam');

## Webcrawling
Do some webcrawling to resolve links

In [ ]:
!pip3 install trafilatura
!pip3 install tika

In [ ]:
import urllib.request
def download_file(download_url, filename):
    response = urllib.request.urlopen(download_url)    
    file = open(filename + ".pdf", 'wb')
    file.write(response.read())
    file.close()

In [ ]:
import trafilatura , re , os
from tika import parser

with open('roam_research.txt') as f:
    text = f.read()

href_regex = r'https?://\S+\)' # find URLs
urls = re.findall(href_regex, text)
urls = [url[:-1] for url in urls]
for url in urls:
    fName = "webcrawl/" + url.replace('https','').replace('http','').replace('://','').replace('/','-')[:30] + ".txt"
    if not os.path.exists(fName):
        try:
            tFile = open(fName , "a")
            if ".pdf" not in url:
                downloaded = trafilatura.fetch_url(url)
                result = trafilatura.extract(downloaded, include_comments=False, include_tables=False, no_fallback=True)
            else:
                download_file(url, "sample")
                raw = parser.from_file('sample.pdf')
                result = raw['content']
            if not result is None:
                tFile.write(result)
            tFile.close()
        except:
            print('oops') # master-level error handling


In [ ]:
#remove empty lines
import os
fList = os.listdir('webcrawl')
cc = 0
for file in fList:
    print(cc)
    cc = cc + 1
    os.system("awk ' /^$/ { print; } /./ { printf(\"%s \", $0); } ' webcrawl/" + file + " > webcrawl/tmp.txt")
    os.system("mv webcrawl/tmp.txt webcrawl/" + file )

In [31]:
#split files
fList = os.listdir('webcrawl/')
os.system('mkdir webcrawl/splitcrawl')
for file in fList:
    os.system('split -l 100 webcrawl/' + file +' webcrawl/splitcrawl/split'+ file[:-4] + '.txt');

## WhatsApp
Whatsapp already comes separated by user - but still worth it to chunk it further

In [ ]:
fList = os.listdir('whatsapp/')
os.system('mkdir whatsapp/splitwhatsapp')
for file in fList:
    os.system('mv whatsapp/' + file.replace(' ','\ ') + ' whatsapp/' + file.replace(' ',''))
    # The name that people have on Whatsapp on my phone is often different from the one in Roam/Email.
    # It's a good idea to make these the same so that the language model can identify them.
    os.system("sed -i '' 's/WHATSAPP_NAME/REAL_NAME/g' whatsapp/" + file) # repeat as often as necessary
    os.system('split -l 400 whatsapp/' + file +' whatsapp/splitwhatsapp/split'+ file[:-4]);

## Facebook
Facebook data just need ~~light~~ massive massaging and splitting

In [ ]:
os.system("sed 's/{//g' facebook.txt > facebook_stripped.txt")
os.system("sed -i '' 's/}//g' facebook_stripped.txt")
os.system("sed -i '' 's/\"//g' facebook_stripped.txt")
os.system("sed -i '' 's/  name: //g' facebook_stripped.txt")
os.system("sed -i '' 's/  message: //g' facebook_stripped.txt")
os.system("sed -i '' '/^$/d' facebook_stripped.txt")
os.system("sed -i '' 's/,$/: /g' facebook_stripped.txt")
os.system("cat facebook_stripped.txt |  paste -d ' ' - - | tee facebook_stripped_full.txt")
os.system("mv facebook_stripped_full.txt facebook_stripped.txt")
os.system("sed -i '' 's/  / /g' facebook_stripped.txt")
# here again you can replace Facebook names with real names
os.system("sed -i '' 's/FACEBOOK_NAME/REAL_NAME/g' facebook_stripped.txt")
os.system("mkdir facebook")
os.system('split -l 100 facebook_stripped.txt facebook/splitfacebook');
os.system("rm facebook_stripped.txt")

## Email
email also needs a lot of massaging after extraction

In [ ]:
import mailbox
mbox_obj = mailbox.mbox('Sent.mbox', create=False)

In [ ]:
# https://stackoverflow.com/questions/7166922/extracting-the-body-of-an-email-from-mbox-file-decoding-it-to-plain-text-regard
def getcharsets(msg):
    charsets = set({})
    for c in msg.get_charsets():
        if c is not None:
            charsets.update([c])
    return charsets

def handleerror(errmsg, emailmsg,cs):
    print('oops') # yes, I feel bad about this too. 

def getbodyfromemail(msg):
    body = None
    #Walk through the parts of the email to find the text body.    
    if msg.is_multipart():    
        for part in msg.walk():
            # If part is multipart, walk through the subparts.            
            if part.is_multipart(): 
                for subpart in part.walk():
                    if subpart.get_content_type() == 'text/plain':
                        # Get the subpart payload (i.e the message body)
                        body = subpart.get_payload(decode=True) 
                        #charset = subpart.get_charset()
            # Part isn't multipart so get the email body
            elif part.get_content_type() == 'text/plain':
                body = part.get_payload(decode=True)
                #charset = part.get_charset()

    # If this isn't a multi-part message then get the payload (i.e the message body)
    elif msg.get_content_type() == 'text/plain':
        body = msg.get_payload(decode=True) 

   # No checking done to match the charset with the correct part. 
    for charset in getcharsets(msg):
        try:
            body = body.decode(charset)
        except UnicodeDecodeError:
            handleerror("UnicodeDecodeError: encountered.",msg,charset)
        except AttributeError:
             handleerror("AttributeError: encountered" ,msg,charset)
    return body    

In [ ]:
import re

for thisemail in mbox_obj:
    body = getbodyfromemail(thisemail)
    if thisemail['To'] is None:
        thisemail['To'] = 'many'
    filename = 'email/' + thisemail['Date'] +  thisemail['To'] 
    filename = filename[:50] + '.txt'
    if not body is None:
        stripped_body = re.sub(r'From:.*' , '' , body); # remove all the header of the email - boring & repetitive!
        stripped_body = re.sub(r'Sent:.*' , '' , stripped_body);
        stripped_body = re.sub(r'To:.*' , '' , stripped_body);
        stripped_body = re.sub(r'Subject.*' , '' , stripped_body);
        stripped_body = re.sub(r'Von:.*' , '' , body); # and then the same thing in German
        stripped_body = re.sub(r'Gesendet:.*' , '' , body);
        stripped_body = re.sub(r'An:.*' , '' , body);
        stripped_body = re.sub(r'Betreff:.*' , '' , body);
        stripped_body = re.sub(r'<.*>' , '' , stripped_body);
        stripped_body = re.sub(r'>' , '' , stripped_body);
        stripped_body = re.sub(r'Cc:.*' , '' , stripped_body);
        stripped_body = re.sub(r'________________________________' , '' , stripped_body);
        stripped_body = os.linesep.join([s for s in stripped_body.splitlines() if s])
        try:
            with open(filename.replace(' ' , ''), 'w') as f:
                f.write(stripped_body)
        except:
            continue

## Telegram
Telegram needs to be queried over the API

In [ ]:
!pip install telethon

In [ ]:
# get your API Key here: https://core.telegram.org/api/obtaining_api_id
from telethon import TelegramClient
from telethon.tl.types import PeerUser, PeerChat, PeerChannel
import os
client = TelegramClient('API_NAME', API_ID, 'API_KEY')
await client.start()

In [ ]:
async for dialog in client.iter_dialogs():
    print(dialog.name, 'has ID', dialog.id)
    if not os.path.isfile('telegram/' + dialog.name.replace('/','') + '.txt' ):
        f = open('telegram/' + dialog.name.replace('/','') + '.txt' , 'w+');
        async for message in client.iter_messages(dialog.id):
            try:
              if not (message.from_id is None):
                  my_user    = await client.get_entity(message.from_id)
                  f.write(my_user.first_name +  ' : ' + message.text)
            except:
                continue
        f.close();

In [ ]:
fList = os.listdir('telegram/')
os.system('mkdir telegram/splittelegram')
for file in fList:
    os.system('mv telegram/' + file.replace(' ','\ ') + ' telegram/' + file.replace(' ',''))
    # again do that thing with the names
    os.system("sed -i '' 's/TELEGRAM_NAME/REAL_NAME/g' telegram/" + file.replace(' ',''))
    os.system('split -l 100 telegram/' + file.replace(' ','') +' telegram/splittelegram/split'+ file[:-4].replace(' ',''));

## epubs
convert epubs to txt and split them

In [ ]:
import os
fList = os.listdir('epubs')
for file in fList:
    os.system('pandoc epubs/' + file.replace(' ' , '\ ') + ' -t plain -o epubs_txt/' + file[:-4].replace(' ' , '\ ') + 'txt')
    os.system("awk ' /^$/ { print; } /./ { printf(\"%s \", $0); } ' epubs_txt/" + file[:-4].replace(' ' , '\ ') + "txt > epubs_txt/tmp.txt")
    os.system("mv epubs_txt/tmp.txt epubs_txt/" + file[:-4].replace(' ' , '\ ') + 'txt')

In [ ]:
fList = os.listdir('epubs_txt/')
os.system('mkdir epubs_txt/splitepubs')
for file in fList:
    os.system('split -l 100 epubs_txt/' + file +' epubs_txt/splitepubs/split'+ file[:-4]);

## docx
convert random collection of .docx and .tex to txt and split

In [ ]:
import os
fList = os.listdir('my_docs')
for file in fList:
    os.system('pandoc my_docs/' + file.replace(' ' , '\ ') + ' -t plain -o my_txt/' + file[:-4].replace(' ' , '\ ') + 'txt')
    os.system("awk ' /^$/ { print; } /./ { printf(\"%s \", $0); } ' my_txt/" + file[:-4].replace(' ' , '\ ') + "txt > my_txt/tmp.txt")
    os.system("mv my_txt/tmp.txt my_txt/" + file[:-4].replace(' ' , '\ ') + 'txt')

In [ ]:
fList = os.listdir('my_txt/')
os.system('mkdir my_txt/splitmytxt')
for file in fList:
    os.system('split -l 100 my_txt/' + file +' my_txt/splitmytxt/split'+ file[:-4]);